In [2]:
## Packages
'''Import all the necessary packages and modules'''
#uses matplotlib qt4 interface
from scipy.fftpack import fftn, ifftn
import hyperspy.api as hs #call hyperspy by hs
hs.preferences.gui(toolkit="traitsui") #opens preference window to change display outputs  
import scipy as sp
import numpy as np # package for playing with matrices
import sys, os # python modules for interacting with computer
from tkinter import filedialog # interface for opening a file explorer
from tkinter import * # * imports all files in tkinter 

ImportError: No module named 'PyQt4'

In [2]:
## Data Directory GUI
'''Getting data directory so data can easily be imported 
using this format data = hs.load('*.dm3', stack=True) to import all dm3s in folder as a stack'''


root = Tk() # Tk() is a function in tkinter that opens a window
root.directory = filedialog.askdirectory() # opens explorer window so you can find the folder of choice
root.withdraw() # closes the tkinter window since it's unnecessary
oldcwd = os.getcwd() # saves old called working directory (place where data is drawn from) as oldcwd use os.chdir(oldcwd) to go back
os.chdir(root.directory) # sets new directory
newcwd = os.getcwd() # saves new directory name as newcwd

In [3]:
## Defining Functions
'''List of definitions that are not currently in any module to be imported'''

def shift_image(im, shift=0, interpolation_order=1, fill_value=np.nan):
    '''Shifts an image based on the input shift
    
    Taken from signal2D.py class in hyperspy
    
    '''
    
    
    
    if np.any(shift):
        fractional, integral = np.modf(shift)
        if fractional.any():
            order = interpolation_order
        else:
            # Disable interpolation
            order = 0
        return sp.ndimage.shift(im, shift, cval=fill_value, order=order)
    else:
        return im
    
def fft_correlation(in1, in2, normalize=False):
    """Correlation of two N-dimensional arrays using FFT.

    Adapted from scipy's fftconvolve.

    Parameters
    ----------
    in1, in2 : array
    normalize: bool
        If True performs phase correlation

    """
    s1 = np.array(in1.shape)
    s2 = np.array(in2.shape)
    size = s1 + s2 - 1
    # Use 2**n-sized FFT
    fsize = (2 ** np.ceil(np.log2(size))).astype("int")
    IN1 = fftn(in1, fsize)
    IN1 *= fftn(in2, fsize).conjugate()
    if normalize is True:
        ret = ifftn(np.nan_to_num(IN1 / np.absolute(IN1))).real.copy()
    else:
        ret = ifftn(IN1).real.copy()
    del IN1
    return ret


In [4]:
data = hs.load('*.dm3', stack=True) # loads all dm3 in the folder directory into a stack called data
# s = data.align2D()  # Gets shifts based on fft_correlation
# data = shift_image(data,s)    #Shifts data cells based on s 

In [6]:
img1 = data.data[0,:,:]
img2 = data.data[1,:,:]
corr = fft_correlation(img1,img2, True)
corr = hs.signals.Signal2D(corr)    #Makes signal hyperspy 2Dsignal
corr.plot()    #Generates a hyperspy plot to visualize data - use h to change the threshold

In [10]:
np.__version__
sp.__version__

'0.17.0'